In [2]:
import sklearn.preprocessing as preprocess
import numpy as np
import pandas as pd
from keras import optimizers
import keras.layers as kl
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import spacy
from keras.utils import to_categorical
from spacy.lang.en import English
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_rows', 1000)
from tqdm import tqdm_notebook
from keras.layers import BatchNormalization, Lambda, Concatenate, Dropout, Conv1D, MaxPooling1D, Input, TimeDistributed, Dense, LSTM, RepeatVector, GlobalAveragePooling1D
from keras.models import Model
import pickle
from datetime import datetime
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from AttentionModules import SelfAttention,CrossAttention
import sys,os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [6]:
dnf700 = pd.read_json('evaluation_set/deepnofakes/dnf_700/initial.json')
dnf_eval = pd.read_json('evaluation_set/deepnofakes/evaluation_50.json')
dnf_eval.columns = ['authors', 'evidence', 'headline', 'id', 'reason', 'claims', 'type', 'urls'] 
with open('evaluation_set/deepnofakes/dnf_700/dnf700_sent_array_id.p', 'rb') as fp:
    articles = pickle.load(fp)
with open('evaluation_set/deepnofakes/dnf_700/dnf700_sent_vector_array_id.p', 'rb') as fp:
    article_vectors = pickle.load(fp)
with open('evaluation_set/word_mapping/id_word_mapping.p', 'rb') as fp:
    id_word_mapping = pickle.load(fp)
dnf_eval.keys(), dnf700.keys(), len(articles.keys()), len(article_vectors.keys())

(Index(['authors', 'evidence', 'headline', 'id', 'reason', 'claims', 'type',
        'urls'],
       dtype='object'),
 Index(['authors', 'headline', 'id', 'type', 'urls'], dtype='object'),
 705,
 705)

In [9]:
dnf700.headline.unique().shape

(698,)

In [8]:
dnf_eval.headline.unique().shape

(50,)

In [7]:
display(dnf700[dnf700.headline == 'WIKILEAKS CLAIM: Hillary Gave Weapons To ISIS'])
display(dnf700[dnf700.headline == 'WikiLeaks: Hillary Clinton knew Saudi, Qatar were funding ISIS – but still took their money for Foundation'])
display(dnf_eval[dnf_eval.headline == 'WIKILEAKS CLAIM: Hillary Gave Weapons To ISIS'])
display(dnf_eval[dnf_eval.headline == 'WikiLeaks: Hillary Clinton knew Saudi, Qatar were funding ISIS – but still took their money for Foundation'])

,authors,headline,id,type,urls


,authors,headline,id,type,urls
35,,"WikiLeaks: Hillary Clinton knew Saudi, Qatar were funding ISIS – but still took their money for Foundation",35,questionable,21stcenturywire.com


,authors,evidence,headline,id,reason,claims,type,urls
0,Bushord Washington,[https://www.snopes.com/fact-check/wikileaks-cofirms-hillary-clinton-sold-weapons-to-isis/],WIKILEAKS CLAIM: Hillary Gave Weapons To ISIS,0,Based on Snopes rating 'Mixture' which means 'a claim has significant elements of both truth and falsity to it such that it could not fairly be described by any other rating.',"[Hillary Clinton is being accused of knowingly allowing American weapons into the hands of ISIS terrorists., Assange claims that among the 1,700 emails just released, there is proof that – despite what she told Congress – she knew about the flow of weapons from Qatar to Libya and then to Syrian rebels and terrorists., So, for example, the disastrous, absolutely disastrous intervention in Libya, the destruction of the Gaddafi government, which led to the occupation of ISIS of large segments of that country, weapons flows going over to Syria, being pushed by Hillary Clinton, into jihadists within Syria, including ISIS, that’s there in those emails.]",1,http://thefederalistpapers.org/us/wikileaks-claim-hillary-gave-weapons-to-isis


,authors,evidence,headline,id,reason,claims,type,urls
42,,[https://www.snopes.com/fact-check/wikileaks-cofirms-hillary-clinton-sold-weapons-to-isis/],"WikiLeaks: Hillary Clinton knew Saudi, Qatar were funding ISIS – but still took their money for Foundation",42,Based on Snopes rating 'Mixture' which means 'a claim has significant elements of both truth and falsity to it such that it could not fairly be described by any other rating.',"[WikiLeaks reveals an early 2014 email where the outgoing Secretary of State Hillary Clinton was urging John Podesta , then advisor to President Barack Obama, to “bring pressure” on Gulf states Saudi Arabia and Qatar, “which are providing clandestine financial and logistic support to ISIL Islamic State, IS, ISIS] and other radical Sunni groups.”, This constitutes proof that the Clinton Foundation knowingly accepting millions of dollars in ‘charitable’ donations from the same Gulf states which both Secretary Clinton and President Obama knew were funding ISIS, Al Nusra Front (al Qaeda in Syria) and known takfiri terrorist fighting organizations., In addition, the Clinton’s Foundation took money from a number of other gulf monarchies: \r\r\nThis latest news validates what 21WIRE has been saying since the ‘ISIS crisis’ began in June 2014.]",1,21stcenturywire.com


In [59]:
articles[62][:5],articles[192][:5]

(['The world’s most reactionary regime, the head-chopping, terror-sponsoring Saudi Arabian kleptocracy, was awarded the chair of the UN Human Rights Council, while Russia has been kicked out.',
  'The travesty was engineered by the Superpower of Lies to punish Moscow for resisting the U.S.-led war of sectarian massacre and regime change in Syria.',
  'The War Party is on the march, to the cheers of corporate media – and Hillary hasn’t even been elected yet.',
  'All attempts to stop the fighting were rejected by the U.S. and NATO and sealed the fate of the Syrian people.”',
  'Did Russia invade Iraq and kill one million people?'],
 ['The world’s most reactionary regime, the head-chopping, terror-sponsoring Saudi Arabian kleptocracy, was awarded the chair of the UN Human Rights Council, while Russia has been kicked out.',
  'The travesty was engineered by the Superpower of Lies to punish Moscow for resisting the U.S.-led war of sectarian massacre and regime change in Syria.',
  'The War

In [28]:

hl = 'WikiLeaks: Hillary Clinton knew Saudi, Qatar were funding ISIS – but still took their money for Foundation '
dnf_eval[dnf_eval.headline==hl]

,authors,evidence,headline,id,reason,claims,type,urls


In [45]:
dnf_eval.drop(columns=['claims'])

,authors,evidence,headline,id,reason,type,urls
0,Bushord Washington,[https://www.snopes.com/fact-check/wikileaks-cofirms-hillary-clinton-sold-weapons-to-isis/],WIKILEAKS CLAIM: Hillary Gave Weapons To ISIS,0,Based on Snopes rating 'Mixture' which means 'a claim has significant elements of both truth and falsity to it such that it could not fairly be described by any other rating.',1,http://thefederalistpapers.org/us/wikileaks-claim-hillary-gave-weapons-to-isis
1,Mike Adams,[https://www.snopes.com/fact-check/clinton-secret-earpiece-debate/],Hillary Clinton Wore Secret Earpiece During First Presidential Debate?,1,Based on Snopes rating 'False' which means 'the primary elements of a claim are demonstrably false.',0,http://www.naturalnews.com/055233_ear_piece_Hillary_Clinton_presidential_debate.html
2,the Burrard Street Journal,"[https://www.snopes.com/fact-check/obama-confirms-he-will-refuse-to-leave-office/, https://www.truthorfiction.com/president-obama-will-refuse-leave-office-trump-elected/]",President Obama Confirms He Will Refuse to Leave Office If Trump Is Elected,2,Based on Snopes rating 'False' which means 'the primary elements of a claim are demonstrably false.',0,http://www.burrardstreetjournal.com/obama-refusing-to-leave-if-trump-elected/
3,admin,"[https://www.snopes.com/fact-check/clinton-votes-found-in-warehouse/, https://local12.com/news/local/ohio-secretary-of-state-debunks-false-story-about-fake-clinton-votes, https://www.truthorfiction.com/thousands-of-fraudulent-clinton-votes-found-in-ohio-warehouse/]",BREAKING: Fraudulent Clinton Votes Discovered By The “Tens Of Thousands”,3,Based on Snopes rating 'False' which means 'the primary elements of a claim are demonstrably false.',0,https://www.usasupreme.com/fbi-director-received-millions-clinton-foundation-brothers-law-firm-clintons-taxes/
4,Alex D.,[https://www.snopes.com/fact-check/comey-clinton-foundation/],"FBI director received millions from Clinton Foundation, his brother’s law firm does Clinton’s taxes",4,"Based on Snopes rating 'Unproven' which means 'insufficient evidence exists to establish the given claim as true, but the claim cannot be definitively proved false.'",1,https://www.usasupreme.com/fbi-director-received-millions-clinton-foundation-brothers-law-firm-clintons-taxes/
5,Paul Joseph Watson,"[https://www.snopes.com/fact-check/hillary-clinton-secret-earpiece/, https://www.snopes.com/fact-check/clinton-secret-earpiece-debate/]",Hillary Clinton Wore 'Secret Earpiece' During Commander-in-Chief Forum,5,"The key claim is false (based on Snopes rating), however, the article also contains opinions stated as fact.",2,http://www.infowars.com/was-hillary-wearing-an-earpiece-during-last-nights-presidential-forum/
6,Baltimore Gazette,"[https://www.snopes.com/fact-check/clinton-received-debate-questions-week-before-debate/, https://www.truthorfiction.com/hillary-clinton-received-debate-questions-advance/]",Clinton Received Debate Questions Week Before Debate,6,Based on Snopes rating 'False' which means 'the primary elements of a claim are demonstrably false.',0,http://www.freemarketcentral.com/index.php/post/2503/report-clinton-received-debate-questions-a-week-before-debate
7,Jim Hoft,[https://www.snopes.com/fact-check/debate-secret-hand-signals/],Hillary Clinton Used Hand Signals to Rig Debate?,7,Based on Snopes rating 'False' which means 'the primary elements of a claim are demonstrably false.',0,http://www.thegatewaypundit.com/2016/09/breaking-poker-pro-agrees-hillary-sending-hand-signals-debate-moderator-holt/
8,Alex Griswold,[https://www.snopes.com/fact-check/hillary-clinton-donations-clinton-foundation/],Hillary Clinton Cut Her Tax Bill by 'Donating' $1 Million to Herself via the Clinton Foundation?,8,Based on Snopes rating 'False' which means 'the primary elements of a claim are demonstrably false.',0,http://www.mediaite.com/election-2016/96-of-the-clintons-2015-charitable-donations-were-to-the-clinton-foundation/
9,A. Michael Smith,[https://www.snopes.c

In [46]:
dnf300.sort_values(by='id')

,authors,evidence,headline,id,reason,type,urls
0,Bushord Washington,[https://www.snopes.com/fact-check/wikileaks-cofirms-hillary-clinton-sold-weapons-to-isis/],WikiLeaks CONFIRMS Hillary Sold Weapons to ISIS... Then Drops Another BOMBSHELL! Breaking News,0,Based on Snopes rating 'Mixture' which means 'a claim has significant elements of both truth and falsity to it such that it could not fairly be described by any other rating.',1,http://thefederalistpapers.org/us/wikileaks-claim-hillary-gave-weapons-to-isis
1,Mike Adams,[https://www.snopes.com/fact-check/clinton-secret-earpiece-debate/],Hillary Clinton Wore Secret Earpiece During First Presidential Debate?,1,Based on Snopes rating 'False' which means 'the primary elements of a claim are demonstrably false.',0,http://www.naturalnews.com/055233_ear_piece_Hillary_Clinton_presidential_debate.html
2,the Burrard Street Journal,"[https://www.snopes.com/fact-check/obama-confirms-he-will-refuse-to-leave-office/, https://www.truthorfiction.com/president-obama-will-refuse-leave-office-trump-elected/]",President Obama Confirms He Will Refuse to Leave Office If Trump Is Elected,2,Based on Snopes rating 'False' which means 'the primary elements of a claim are demonstrably false.',0,http://www.burrardstreetjournal.com/obama-refusing-to-leave-if-trump-elected/
3,admin,"[https://www.snopes.com/fact-check/clinton-votes-found-in-warehouse/, https://local12.com/news/local/ohio-secretary-of-state-debunks-false-story-about-fake-clinton-votes, https://www.truthorfiction.com/thousands-of-fraudulent-clinton-votes-found-in-ohio-warehouse/]",BREAKING: Fraudulent Clinton Votes Discovered By The “Tens Of Thousands”,3,Based on Snopes rating 'False' which means 'the primary elements of a claim are demonstrably false.',0,https://www.usasupreme.com/fbi-director-received-millions-clinton-foundation-brothers-law-firm-clintons-taxes/
4,Alex D.,[https://www.snopes.com/fact-check/comey-clinton-foundation/],"FBI director received millions from Clinton Foundation, his brother’s law firm does Clinton’s taxes",4,"Based on Snopes rating 'Unproven' which means 'insufficient evidence exists to establish the given claim as true, but the claim cannot be definitively proved false.'",1,https://www.usasupreme.com/fbi-director-received-millions-clinton-foundation-brothers-law-firm-clintons-taxes/
5,Paul Joseph Watson,"[https://www.snopes.com/fact-check/hillary-clinton-secret-earpiece/, https://www.snopes.com/fact-check/clinton-secret-earpiece-debate/]",Hillary Clinton Wore 'Secret Earpiece' During Commander-in-Chief Forum,5,"The key claim is false (based on Snopes rating), however, the article also contains opinions stated as fact.",2,http://www.infowars.com/was-hillary-wearing-an-earpiece-during-last-nights-presidential-forum/
6,Baltimore Gazette,"[https://www.snopes.com/fact-check/clinton-received-debate-questions-week-before-debate/, https://www.truthorfiction.com/hillary-clinton-received-debate-questions-advance/]",Clinton Received Debate Questions Week Before Debate,6,Based on Snopes rating 'False' which means 'the primary elements of a claim are demonstrably false.',0,http://www.freemarketcentral.com/index.php/post/2503/report-clinton-received-debate-questions-a-week-before-debate
7,Jim Hoft,[https://www.snopes.com/fact-check/debate-secret-hand-signals/],Hillary Clinton Used Hand Signals to Rig Debate?,7,Based on Snopes rating 'False' which means 'the primary elements of a claim are demonstrably false.',0,http://www.thegatewaypundit.com/2016/09/breaking-poker-pro-agrees-hillary-sending-hand-signals-debate-moderator-holt/
8,Alex Griswold,[https://www.snopes.com/fact-check/hillary-clinton-donations-clinton-foundation/],Hillary Clinton Cut Her Tax Bill by 'Donating' $1 Million to Herself via the Clinton Foundation?,8,Based on Snopes rating 'False' which means 'the primary elements of a claim are demonstrably false.',0,http://www.mediaite.com/election-2016/96-of-the-clintons-2015-charitable-donations-were-to-the-clinton-fou

In [66]:
titles = sorted(dnf300.headline.unique())

In [67]:
test_titles = sorted(dnf_eval.headline.unique())

In [68]:
titles = sorted(dnf300.headline.unique())
non_test_titles = list(set(titles)-set(test_titles))
val_titles = non_test_titles[200:]
train_titles = titles[:200]
len(train_titles),len(val_titles),len(test_titles)

(200, 50, 50)

In [69]:
train_batchsize = 32
val_batchsize = 32
test_batchsize = 50
train_steps_per_epoch = 4
val_steps_per_epoch = 1
epochs = 50000

In [12]:
def load_spacy():
    sentencizer = English()
    sentencizer.add_pipe(sentencizer.create_pipe('sentencizer'))
    nlp = spacy.load("en_core_web_md")
    return sentencizer, nlp
sentencizer, nlp = load_spacy()


In [70]:
def datagen_dnf(batchsize,dataframe,mode):
    counter=0
    ar_ids,ar_sents,ar_head_vectors,ar_head_classes,hds=[],[],[],[],[]
    while True:
        if mode=='train':
            idx=np.random.choice(train_titles)
        elif mode=='val':
            idx=np.random.choice(val_titles)
        elif mode=='test':
            idx=np.random.choice(test_titles)
        idx = idx.strip()
        try:
            hd = dataframe[dataframe.headline==idx]['headline'].values[0].lower()
            hds.append(hd)
            ar_id = dataframe[dataframe.headline==idx]['id'].values[0]
        except Exception as ex:
            print(idx)
#         cl = dataframe[dataframe.Article==idx]['Claim'].values
#         sentences=articles[ar_id]
#         print(len(sentences))
        sents = np.zeros((1300,300))
        vectors = article_vectors[ar_id]
        sents[:len(vectors)] = vectors
        ar_ids.append(ar_id)
        ar_sents.append(sents)
        hd_nlp = nlp(hd.lower())
        head_classes = np.zeros(50, dtype='int')
        for i in range(len(hd_nlp)):
            head_classes[i] = hd_nlp[i].rank
        ar_head_vectors.append(hd_nlp.vector)
        ar_head_classes.append(to_categorical(num_classes=20000,y=head_classes))
        counter+=1
        if counter==batchsize:
            inputs = {
                'article_id': np.array(ar_ids)
                ,'headline': np.array(hds)
                ,'sentence_vectors' : np.array(ar_sents)
                ,'headline_vector': np.array(ar_head_vectors)
            }
            outputs = {
                'headline_token_classes': np.array(ar_head_classes)
            }
            yield inputs,outputs
            ar_ids,ar_sents,ar_head_vectors,ar_head_classes,hds=[],[],[],[],[]
            counter=0
 

In [71]:
tdg = datagen_dnf(train_batchsize,dnf300,mode='train')
vdg = datagen_dnf(val_batchsize,dnf300,mode='val')
test_dg = datagen_dnf(test_batchsize,dnf300,mode='test')

In [72]:
x,y = next(test_dg)

WIKILEAKS CLAIM: Hillary Gave Weapons To ISIS


In [ ]:
x['sentence_vectors'].shape, x['headline_vector'].shape, y['headline_token_classes'].shape

In [ ]:
def build_model():
    inp_sentence_vectors = Input(shape=(1300, 300), name='sentence_vectors')
    inp_headline_vector = Input(shape=(300,), name='headline_vector')
    conv1 = Conv1D(filters=16,kernel_size=3,strides=1,activation='relu', padding='same')(inp_sentence_vectors)
    conv1 = Dropout(0.5)(conv1)
    conv2 = Conv1D(filters=32,kernel_size=3,strides=1,activation='relu', padding='same')(conv1)
    conv2 = Dropout(0.5)(conv2)
    conv2 = BatchNormalization()(conv2)
    sent_sa_feat_1, sent_beta_1, sent_gamma_1 = SelfAttention(int(conv2.shape[-1]))(conv2)
    sent_sa_feat_2, sent_beta_2, sent_gamma_2 = SelfAttention(int(conv2.shape[-1]))(conv2)
    sent_sa_feat_3, sent_beta_3, sent_gamma_3 = SelfAttention(int(conv2.shape[-1]))(conv2)
    sent_sa_feat_4, sent_beta_4, sent_gamma_4 = SelfAttention(int(conv2.shape[-1]))(conv2)
    concat1 = Concatenate()([sent_sa_feat_1,sent_sa_feat_2,sent_sa_feat_3,sent_sa_feat_4])
    conv3 = Conv1D(filters=256,kernel_size=3, strides=1, activation='relu', padding='same')(concat1)
    conv3 = Dropout(0.5)(conv3)
    conv3 = BatchNormalization()(conv3)
    headline = Dense(256, activation='relu')(inp_headline_vector)
    headline = Lambda(lambda x:K.expand_dims(x, axis=1))(headline)
    headline = BatchNormalization()(headline)
    sent_hd_sa_feat_1, sent_hd_beta_1, sent_hd_gamma_1 = CrossAttention(int(conv3.shape[-1]))([headline,conv3])
    sent_hd_sa_feat_2, sent_hd_beta_2, sent_hd_gamma_2 = CrossAttention(int(conv3.shape[-1]))([headline,conv3])
    sent_hd_sa_feat_3, sent_hd_beta_3, sent_hd_gamma_3 = CrossAttention(int(conv3.shape[-1]))([headline,conv3])
    sent_hd_sa_feat_4, sent_hd_beta_4, sent_hd_gamma_4 = CrossAttention(int(conv3.shape[-1]))([headline,conv3])  
    concat3 = Concatenate()([sent_hd_sa_feat_1,sent_hd_sa_feat_2,sent_hd_sa_feat_3,sent_hd_sa_feat_4])
    concat3 = Dropout(0.5)(concat3)
    concat3 = BatchNormalization()(concat3)
    conv5 = Conv1D(filters=256,kernel_size=3, strides=2, activation='relu', padding='same')(concat3)
    conv6 = Conv1D(filters=256,kernel_size=3, strides=2, activation='relu', padding='same')(conv5)
    conv7 = Conv1D(filters=256,kernel_size=3, strides=2, activation='relu', padding='same')(conv6)
    conv8 = Conv1D(filters=256,kernel_size=3, strides=2, activation='relu', padding='same')(conv7)
    conv8 = Dropout(0.5)(conv8)
    conv8 = BatchNormalization()(conv8)
    gap = GlobalAveragePooling1D()(conv8)
    repeat = RepeatVector(50)(gap)
    lstm1 = LSTM(256,return_sequences=True, activation='relu')(repeat)
    lstm1 = Dropout(0.5)(lstm1)
    lstm1 = BatchNormalization()(lstm1)
    gen_hd_vector = TimeDistributed(Dense(20000,activation='softmax'), name='headline_token_classes')(lstm1)
    model = Model([inp_sentence_vectors,inp_headline_vector],gen_hd_vector)
    return model
model = build_model()
model.compile(optimizer=optimizers.Adam(lr=0.0001,beta_1=0.0,beta_2=0.99),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
# print('model params:',model.count_params())
SVG(model_to_dot(model,show_layer_names=True,show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
dt = datetime.now()
mc = ModelCheckpoint('weights/cdc_sa_sent_hd_word.hdf5',save_best_only=True,save_weights_only=True)
tb = TensorBoard(batch_size=32,log_dir='logs/cdc_sa_sent_hd_word/{0}'.format(dt.timestamp()),write_graph=True)

In [ ]:
hist = model.fit_generator(tdg, callbacks=[mc,tb], initial_epoch=13420
                           ,steps_per_epoch=train_steps_per_epoch
                           ,validation_data=vdg
                           ,validation_steps=val_steps_per_epoch
                           ,epochs=epochs)

In [ ]:
# from matplotlib import pyplot as plt
# acc = hist.history['acc']
# loss = hist.history['loss']

# # Create count of the number of epochs
# epoch_count = range(1, len(acc) + 1)

# # Visualize loss history
# # plt.plot(epoch_count, acc, 'b-')
# fig, ax = plt.subplots(ncols=2,sharex=True)
# ax[0].plot(epoch_count, loss, 'r--')
# ax[0].legend(['Loss'])
# ax[0].set_xlabel('Epoch')
# ax[0].set_ylabel('Loss')
# ax[1].plot(epoch_count, acc, 'b-')
# ax[1].legend(['Accuracy'])
# ax[1].set_xlabel('Epoch')
# ax[1].set_xlabel('Accuracy')
# plt.show();

In [ ]:
# df_res = pd.DataFrame(hist.history)

In [ ]:
# df_res[['loss','val_loss']].plot()

In [ ]:
# df_res[['acc','val_acc']].plot()

# Evalutate SST

In [ ]:
model.load_weights('weights/cdc_sa_sent_hd_word.hdf5')
model.evaluate_generator(test_dg,steps=1)

In [ ]:
x,y = next(test_dg)
pred = model.predict(x)

In [ ]:
x

In [ ]:
pred_classes = np.argmax(pred[0], axis=-1)

In [ ]:
pred_classes

In [ ]:
doc = nlp(str(x['headline'][0]))
doc.text

In [ ]:
ids = []
for token in doc:
    if token.has_vector:
#         print(token.norm)
        id = nlp.vocab.vectors.key2row[token.norm]
    else:
        id = None
    ids.append(id)

print([token for token in doc])
print(ids)

In [ ]:
words = []
for idx in pred_classes:
    if idx!=0:
        words.append(id_word_mapping[idx])
print(words)

In [ ]:
# df_key_value = pd.DataFrame(nlp.vocab.vectors.key2row,index=[0]).T.reset_index()
# df_key_value.columns = ['key','value']

In [ ]:
# df_key_value[df_key_value['value']==1282]['key'].values[0]

In [ ]:
# nlp.vocab.vectors.data[1282]

In [ ]:
# len(np.unique(list(nlp.vocab.vectors.key2row.keys()))),len(np.unique(list(nlp.vocab.vectors.key2row.values())))

In [ ]:
# vocab_dict = {}

In [ ]:
# counter = 0
# for s in nlp.vocab.strings:
#     print(s.
#     counter+=1
#     if counter==100:
#         break
    

In [ ]:


#     else:
#         print(len(df_ar_cl_cdc[df_ar_cl_cdc.Article==ar]['Claim'].values))

In [ ]:
# df_ar_cl_cdc.head()

In [ ]:
# sorted(df_hd_tp_cdc.Title.unique())

In [ ]:
# sorted(df_ar_cl_cdc.Article.unique())

In [ ]:
# for token1,token2 in zip(nlp('how are you hillary!'),nlp('How are You HILLARY!')):
#     print(token1.rank,token2.rank)

In [ ]:

from keras import backend as K
from keras.layers import Layer,InputSpec,Conv1D 

import tensorflow as tf

class SelfAttention(Layer):
    def __init__(self, ch, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.channels = ch
        self.filters_q_k = self.channels // 8
        self.filters_v = self.channels
#         print(self.channels,self.filters_f_g,self.filters_h)

    def build(self, input_shape):
        self.N = input_shape[1]        
        self.gamma = self.add_weight(name='gamma', shape=[1], initializer='zeros', trainable=True)

        super(SelfAttention, self).build(input_shape)
        # Set input spec.
        self.input_spec = InputSpec(ndim=3,
                                    axes={2: input_shape[-1]})
        self.built = True


    def call(self, x):
        def hw_flatten(x):
            return K.reshape(x, shape=[K.shape(x)[0], K.shape(x)[1]*K.shape(x)[2], K.shape(x)[-1]])

        q = Conv1D(filters=self.filters_q_k, kernel_size=3, padding='same')(x)
        k = Conv1D(filters=self.filters_q_k, kernel_size=3, padding='same')(x)
        v = Conv1D(filters=self.filters_v, kernel_size=3, padding='same')(x)
#         print('q.shape,k.shape,v.shape,',q.shape,k.shape,v.shape)
        s = tf.matmul(q, k, transpose_b=True)  # # [bs, N, N]
#         print('s.shape:',s.shape)
        beta = K.softmax(s, axis=-1)  # attention map
        self.beta_shape = tuple(beta.shape[1:].as_list())
#         print('beta.shape:',beta.shape.as_list())
        o = K.batch_dot(beta, v)  # [bs, N, C]
#         print('o.shape:',o.shape)
#         o = K.reshape(o, shape=K.shape(x))  # [bs, h, w, C]
        x = self.gamma * o + x
#         print('x.shape:',x.shape)
        return [x, beta, self.gamma]

    def compute_output_shape(self, input_shape):
        return [input_shape,self.beta_shape, tuple(self.gamma.shape.as_list())]

In [ ]:
 (1, 1) + (32, 32//8)

In [ ]:
q = Conv1D(filters=10, kernel_size=3, padding='same')

In [ ]:
t = q(Input(shape=(32,3))).shape

In [ ]:
tuple(t.as_list())

In [10]:
" sdfsd sdfsd sdff    ".strip()

'sdfsd sdfsd sdff'